<div style='background-image: url("../../share/images/header.svg") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Bases de datos en R</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">Ulises M. Alvarez</div>
        </div>
    </div>
</div>

## Porqué usar bases de datos

1. Podemos interrogar nuestros datos (hacerles preguntas).
2. Podemos buscar datos con relativa rapidez.
3. Podemos relacionar los datos de dos tablas diferentes utilizando *JOINs*.
4. Podemos crear informes significativos de nuestros datos.
5. Los datos tienen una estructura incorporada.
6. La información de un tipo, cualquiera que este sea, sólo se almacena una única vez.
7. Las DB son ACID (acrónimo de *Atomicity, Consistency, Isolation and Durability*)
8. Las DB son tolerantes a fallas.
9. Las DB pueden manejar grandes conjuntos de datos.
10. Las DB son concurrentes, esto es, múltiples usuarios pueden acceder al mismo tiempo a los datos sin corromperlos.
11. Las DB son escalables.

**Fuente**
* https://softwareengineering.stackexchange.com/questions/190482/why-use-a-database-instead-of-just-saving-your-data-to-disk

### *ACID (Atomicity, Consistency, Isolation and Durability)*


#### Atomicidad.
Si cuando una operación consiste en una serie de pasos, bien todos ellos se ejecutan o bien ninguno, es decir, las transacciones son completas.

#### Consistencia (integridad). 
Es la propiedad que asegura que sólo se empieza aquello que se puede acabar. Por lo tanto se ejecutan aquellas operaciones que no van a romper las reglas y directrices de Integridad de la base de datos. 

#### Aislamiento.
Esta propiedad asegura que una operación no puede afectar a otras. 

#### Durabilidad (persistencia). 
Esta propiedad asegura que una vez realizada la operación, ésta persistirá y no se podrá deshacer aunque falle el sistema.

**Fuente:**
* https://es.wikipedia.org/wiki/ACID

## Bases de datos en R

Vamos a usar DB en R cuando:

* Deseamos trabajar con los datos de una base sin exportarlos, o
* Tenemos un conjunto de datos que no cabe en la RAM de nuestro equipo.


In [1]:
options(repos = "https://cloud.r-project.org")      # Definimos repositorio 
if (!require("pacman")) install.packages("pacman")  # Si no tenemos, pacman, lo instalamos

Loading required package: pacman


In [2]:
pacman::p_load(dplyr,                               # librería para manipular datos
              RSQLite,                              # librería para acceder a SQLite
              nycflights13)                         # data for all, 336776, flights departing NYC in 2013

In [18]:
my_db <- src_sqlite("my_db.sqlite3", create = T)    # just give the path and the ok to create a table.

Ahora, copiaremos los datos mediante:

```r
copy_to()
```

> El uso de esta función no es recomendable con grandes conjuntos de datos.

In [20]:
flights_sqlite <- copy_to(my_db,                           # destino
                          flights,                         # DF de origen 
                          temporary = FALSE,               # hacemos nuestro objeto no temporal
                          indexes =                        # Generamos el "index" para acelerar análisis
                          list(c("year", "month", "day"),  
                               "carrier", "tailnum"))

ERROR: Error: Table flights already exists.


In [22]:
flights_sqlite <- tbl(nycflights13_sqlite(), "flights")    # Caching nycflights db

In [24]:
explain(flights_sqlite)

<SQL>
SELECT *
FROM `flights`


<PLAN>
   addr       opcode p1  p2 p3      p4 p5 comment
1     0         Init  0  33  0         00      NA
2     1     OpenRead  0 131  0      19 00      NA
3     2       Rewind  0  31  0         00      NA
4     3       Column  0   0  1         00      NA
5     4       Column  0   1  2         00      NA
6     5       Column  0   2  3         00      NA
7     6       Column  0   3  4         00      NA
8     7       Column  0   4  5         00      NA
9     8       Column  0   5  6         00      NA
10    9 RealAffinity  6   0  0         00      NA
11   10       Column  0   6  7         00      NA
12   11       Column  0   7  8         00      NA
13   12       Column  0   8  9         00      NA
14   13 RealAffinity  9   0  0         00      NA
15   14       Column  0   9 10         00      NA
16   15       Column  0  10 11         00      NA
17   16       Column  0  11 12         00      NA
18   17       Column  0  12 13         00      NA
19   18    

In [25]:
tbl(my_db, sql("SELECT * FROM flights"))  # Probemos a ejecutar la búsqueda con SQL...

Source:   query [?? x 19]
Database: sqlite 3.11.1 [my_db.sqlite3]

# A tibble: ?? x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     1     1      517            515         2      830            819
 2  2013     1     1      533            529         4      850            830
 3  2013     1     1      542            540         2      923            850
 4  2013     1     1      544            545        -1     1004           1022
 5  2013     1     1      554            600        -6      812            837
 6  2013     1     1      554            558        -4      740            728
 7  2013     1     1      555            600        -5      913            854
 8  2013     1     1      557            600        -3      709            723
 9  2013     1     1      557            600        -3      838            846
10  2013     1     1      558            600

## Acciones básicas

### Select columns with select()

In [26]:
select(flights_sqlite,  # los datos 
       year:day,        # seleccionamos desde la columna 'year', hasta 'day'
       dep_delay,       # también 'dep_delay'
       arr_delay)       # y 'arr_delay'

Source:   query [?? x 5]
Database: sqlite 3.11.1 [/tmp/Rtmplxf76Y/nycflights13.sqlite]

# A tibble: ?? x 5
    year month   day dep_delay arr_delay
   <int> <int> <int>     <dbl>     <dbl>
 1  2013     1     1         2        11
 2  2013     1     1         4        20
 3  2013     1     1         2        33
 4  2013     1     1        -1       -18
 5  2013     1     1        -6       -25
 6  2013     1     1        -4        12
 7  2013     1     1        -5        19
 8  2013     1     1        -3       -14
 9  2013     1     1        -3        -8
10  2013     1     1        -2         8
# ... with more rows

### Filter rows with filter()

In [27]:
filter(flights_sqlite,   # los datos
       dep_delay > 240)  # seleccionamos un retraso mayor a 240...

Source:   query [?? x 19]
Database: sqlite 3.11.1 [/tmp/Rtmplxf76Y/nycflights13.sqlite]

# A tibble: ?? x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     1     1      848           1835       853     1001           1950
 2  2013     1     1     1815           1325       290     2120           1542
 3  2013     1     1     1842           1422       260     1958           1535
 4  2013     1     1     2115           1700       255     2330           1920
 5  2013     1     1     2205           1720       285       46           2040
 6  2013     1     1     2343           1724       379      314           1938
 7  2013     1     2     1332            904       268     1616           1128
 8  2013     1     2     1412            838       334     1710           1147
 9  2013     1     2     1607           1030       337     2003           1355
10  2013     1     2  

### Arrange rows with arrange()

Parecido a *filter*, pero reordena. Si se selecciona más de una columna, cada columna adicional se usará para reordenar la precedente.

In [30]:
arrange(flights_sqlite,    # los datos
        year, month, day)  # las columnas

Source:   query [?? x 19]
Database: sqlite 3.11.1 [/tmp/Rtmplxf76Y/nycflights13.sqlite]

# A tibble: ?? x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     1     1      517            515         2      830            819
 2  2013     1     1      533            529         4      850            830
 3  2013     1     1      542            540         2      923            850
 4  2013     1     1      544            545        -1     1004           1022
 5  2013     1     1      554            600        -6      812            837
 6  2013     1     1      554            558        -4      740            728
 7  2013     1     1      555            600        -5      913            854
 8  2013     1     1      557            600        -3      709            723
 9  2013     1     1      557            600        -3      838            846
10  2013     1     1  

### Add new columns with mutate()

In [31]:
mutate(flights_sqlite,               # los datos
       speed = air_time / distance)  # agregamos la columna 'speed'

Source:   query [?? x 20]
Database: sqlite 3.11.1 [/tmp/Rtmplxf76Y/nycflights13.sqlite]

# A tibble: ?? x 20
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     1     1      517            515         2      830            819
 2  2013     1     1      533            529         4      850            830
 3  2013     1     1      542            540         2      923            850
 4  2013     1     1      544            545        -1     1004           1022
 5  2013     1     1      554            600        -6      812            837
 6  2013     1     1      554            558        -4      740            728
 7  2013     1     1      555            600        -5      913            854
 8  2013     1     1      557            600        -3      709            723
 9  2013     1     1      557            600        -3      838            846
10  2013     1     1  

### Summarise values with summarise()

In [32]:
summarise(flights_sqlite,          # los datos
          delay = mean(dep_time))  # la función a aplicar, mean()

Source:   query [?? x 1]
Database: sqlite 3.11.1 [/tmp/Rtmplxf76Y/nycflights13.sqlite]

# A tibble: ?? x 1
    delay
    <dbl>
1 1349.11

## "Laziness"

When working with databases, dplyr tries to be as lazy as possible:

* It never pulls data into R unless you explicitly ask for it.
*  It delays doing any work until the last possible moment: it collects together everything you want to do and then sends it to the database in one step.

For example, take the following code:

In [13]:
c1 <- filter(flights_sqlite, year == 2013, month == 1, day == 1)
c2 <- select(c1, year, month, day, carrier, dep_delay, air_time, distance)
c3 <- mutate(c2, speed = distance / air_time * 60)
c4 <- arrange(c3, year, month, day, carrier)

Suprisingly, this sequence of operations never actually touches the database. It’s not until you ask for the data (e.g. by printing c4) that dplyr generates the SQL and requests the results from the database. Even then it only pulls down 10 rows.

In [14]:
c4

Source:   query [?? x 8]
Database: sqlite 3.11.1 [/tmp/Rtmplxf76Y/nycflights13.sqlite]

# A tibble: ?? x 8
    year month   day carrier dep_delay air_time distance    speed
   <int> <int> <int>   <chr>     <dbl>    <dbl>    <dbl>    <dbl>
 1  2013     1     1      9E         0      189     1029 326.6667
 2  2013     1     1      9E        -9       57      228 240.0000
 3  2013     1     1      9E        -3       68      301 265.5882
 4  2013     1     1      9E        -6       57      209 220.0000
 5  2013     1     1      9E        -8       66      264 240.0000
 6  2013     1     1      9E         0       40      184 276.0000
 7  2013     1     1      9E         6      146      740 304.1096
 8  2013     1     1      9E         0      139      665 287.0504
 9  2013     1     1      9E        -8      150      765 306.0000
10  2013     1     1      9E        -6       41      187 273.6585
# ... with more rows

To pull down all the results use collect(), which returns a tbl_df():

In [15]:
collect(c4)

year,month,day,carrier,dep_delay,air_time,distance,speed
2013,1,1,9E,0,189,1029,326.6667
2013,1,1,9E,-9,57,228,240.0000
2013,1,1,9E,-3,68,301,265.5882
2013,1,1,9E,-6,57,209,220.0000
2013,1,1,9E,-8,66,264,240.0000
2013,1,1,9E,0,40,184,276.0000
2013,1,1,9E,6,146,740,304.1096
2013,1,1,9E,0,139,665,287.0504
2013,1,1,9E,-8,150,765,306.0000
2013,1,1,9E,-6,41,187,273.6585


You can see the query dplyr has generated by looking at the query component of the object:

In [16]:
c4$query

NULL

You can also ask the database how it plans to execute the query with explain(). The output for SQLite is described in more detail on the SQLite website. It’s helpful if you’re trying to figure out which indexes are being used.

In [17]:
explain(c4)

<SQL>
SELECT *
FROM (SELECT `year`, `month`, `day`, `carrier`, `dep_delay`, `air_time`, `distance`, `distance` / `air_time` * 60.0 AS `speed`
FROM (SELECT `year` AS `year`, `month` AS `month`, `day` AS `day`, `carrier` AS `carrier`, `dep_delay` AS `dep_delay`, `air_time` AS `air_time`, `distance` AS `distance`
FROM (SELECT *
FROM `flights`
WHERE ((`year` = 2013.0) AND (`month` = 1.0) AND (`day` = 1.0)))))
ORDER BY `year`, `month`, `day`, `carrier`


<PLAN>
   addr       opcode p1    p2 p3           p4 p5 comment
1     0         Init  0    56  0              00      NA
2     1   SorterOpen  4     9  0       k(1,B) 00      NA
3     2     OpenRead  3   131  0           19 00      NA
4     3     OpenRead  5 22656  0     k(4,,,,) 02      NA
5     4         Real  0     1  0         2013 00      NA
6     5         Real  0     2  0            1 00      NA
7     6         Real  0     3  0            1 00      NA
8     7     Affinity  1     3  0          DDD 00      NA
9     8       SeekGE  5   